Import essential functions from the hcr probe designer module.

In [2]:
from AGambiaeHCRdesign import read_fasta, create_oligos, blast_oligos, filter_and_rank, add_hairpin

Next, read a FASTA file containing the sequence of the transcript for which HCR probes need to be designed. The file should be in a .fasta or .faa format.

In [ ]:
ID, desc, sequence = read_fasta("test_gene.fasta")

Transcript ID:  AGAP004691.R436
Transcript Description:  AGAP004691.R436 | gene=AGAP004691 | organism=Anopheles_gambiae_PEST | gene_product=LIM domain-binding protein 1 | transcript_product=LIM domain-binding protein 1 | location=AgamP4_2L:1272628-1290444(-) | length=4854 | sequence_SO=chromosome | SO=protein_coding_gene | is_pseudo=false
Transcript Sequence:  AAACGTCCTAGTGTTGACCATATGTGTTCTCTGTGTGTTTCAATGACATATATTGTCTGAACCTTTTTCAAAATGGCTGCCAGTGTTGAAAAAAATTGATGGCGTCTATGTTGGCAAGTCCGCTGTGAGTAAAATTGTAATAATGATAATATGTGAAAATATCATTTGAAACATCCGTGAATGACCATATTTGTACGAATTAATGTTAAATGTGTAATTTAGCGATATTGAGCTATTAGAGGATCAAGCCTCTATATCGGTTCACAGTTCGATAAAAGTTCACGGAGTTTTTGTGATAGTGAAGCGTATGCTTGTTGGCTTCAAAAGAGGTGGGGAGTTTTAGATGTTCGACTAGCTGTTTAACGAACTAGTGATAAAAAGATACAAATCAGTTTAAAGATAAAATAATTGTGACAATTTGAATCGTGTAGAGAATCAATGCTACATGCTGTTATTAAAACCAATAGTACTCCAATAGCCGTTTAGACGTGTGCCGTTTACCGTTTGGTTGTGCTGTGTTGTAAAGGTACGGAGCAGGCTGGCGTACCGCACACGGTGCGGTTGCGCACTTCTAGGATCTAACATTCTTCCTTAGTAGTTGCAATTTTGGAATATGATAGGTTTAAGTCGACGTGGA

Now, the transcript sequence will be tiled into oligos (25 bases in length). 

Optional: You can also adjust the gap between oligos (2 bases by default), and frame start position (0 i.e. beginning of the transcript by default)

In [4]:
oligos_all = create_oligos(sequence,oligo_length=25)

Oligos tiled along the transcript sequence: 

[Seq('ACATATGGTCAACACTAGGACGTTT'), Seq('TATATGTCATTGAAACACACAGAGA'), Seq('CAGCCATTTTGAAAAAGGTTCAGAC'), Seq('GACGCCATCAATTTTTTTCAACACT'), Seq('TTTACTCACAGCGGACTTGCCAACA'), Seq('ATTTTCACATATTATCATTATTACA'), Seq('TGGTCATTCACGGATGTTTCAAATG'), Seq('CACATTTAACATTAATTCGTACAAA'), Seq('CTCTAATAGCTCAATATCGCTAAAT'), Seq('ACTGTGAACCGATATAGAGGCTTGA'), Seq('CACAAAAACTCCGTGAACTTTTATC'), Seq('GAAGCCAACAAGCATACGCTTCACT'), Seq('GAACATCTAAAACTCCCCACCTCTT'), Seq('TATCACTAGTTCGTTAAACAGCTAG'), Seq('TTATCTTTAAACTGATTTGTATCTT'), Seq('TACACGATTCAAATTGTCACAATTA'), Seq('TAATAACAGCATGTAGCATTGATTC'), Seq('AAACGGCTATTGGAGTACTATTGGT'), Seq('CAACCAAACGGTAAACGGCACACGT'), Seq('CCTGCTCCGTACCTTTACAACACAG'), Seq('GCAACCGCACCGTGTGCGGTACGCC'), Seq('AGGAAGAATGTTAGATCCTAGAAGT'), Seq('TATCATATTCCAAAATTGCAACTAC'), Seq('TGCACTAACTCCACGTCGACTTAAA'), Seq('TGTAGACGGTAAACTCAGTGACGGT'), Seq('TTTCCAATTGTTTGTAGAGTCCTCC'), Seq('AGGGTCGGTTGGTCCCGAAGGTGTT'), Seq('TCCTCCAGGACCCGCACCTTGCTGC'), S

Next, perform a batch BLAST of each of the oligos against the transcriptome database (AGambie PEST data available here: https://vectorbase.org/vectorbase/app/downloads/Current_Release/AgambiaePEST/fasta/data) and save the BLAST results.

Warning: BLAST+ must be installed on the computer and a BLAST database should be built using 'makeblastdb' command on the data downloaded from vector base.
This is a pre-requisite for the BLAST function to work.

In [ ]:
blast_result = blast_oligos(oligos_all, "AGambiaePEST")

Running batch BLAST on oligos
BLAST completed successfully


View the BLAST results

In [6]:
blast_result.head()

,qid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,AGAP004691.R437,100.0,25,0,0,1,25,25,1,3.920000e-07,50.1
1,1,AGAP004691.R436,100.0,25,0,0,1,25,25,1,3.920000e-07,50.1
2,1,AGAP009303.R209,100.0,14,0,0,12,25,8733,8720,1.400000e+00,28.2
3,1,AGAP000898-RA,100.0,12,0,0,14,25,9840,9829,2.200000e+01,24.3
4,1,AGAP010249-RA,100.0,12,0,0,2,13,2401,2390,2.200000e+01,24.3


The oligos also need to be filtered based on the GC content and the Melting Temperature. 
Default range for GC: 37-85 % & Default range for MT: 47-85 C
These can also be adjusted in the 'filter and rank' function but is optional. 

Based on the BLAST results, neighbouring oligos that hit the same transcript (other than our target gene) should be filtered out. 
This is to ensure specificity of HCR probes

Lastly, the neighbouring oligo pairs are ranked according to their specificity

In [7]:
probe_datasheet = filter_and_rank(ID, oligos_all, blast_result)

Filtering out oligos based on GC content and Melting Temperature...
Filtering Completed
Filtering out oligos if adjacent oligos have hits on the same transcript...
Fitering Completed
Ranking oligo pairs by specificity
Ranking Completed
Generated number of optimized HCR probe pairs:  26


c:\Users\CTrivedi\anaconda3\envs\hcr\lib\site-packages\Bio\SeqUtils\__init__.py:144: BiopythonDeprecationWarning: GC is deprecated; please use gc_fraction instead.
  warnings.warn(


View filtered results datasheet

In [8]:
probe_datasheet.head()

,Oligo1_Position,Oligo2_Position,Oligo1_Sequence,Oligo2_Sequence,Score (average hits)
0,23,24,TATCATATTCCAAAATTGCAACTAC,TGCACTAACTCCACGTCGACTTAAA,1.0
1,35,36,TAAATTTTGATAACTATTCGAAGCA,TGGCGTATTTGAACCAGTGCCTGGT,1.0
2,41,42,GACAGGGCCGTTGTATGGTCCTCCT,TGGACCTACTGCTACAGGGCCACCT,1.0
3,84,85,ATCTTCATCCCCAAACTCACCTCCC,GTTTTCAAGCCTAGTGATCAATCGT,1.0
4,89,90,TCCTCCGCTCATTGGACCTCCATCA,TCCTCGATCCACCTGCCATGAAGTA,1.0


Choose hairpin (B1-B5) for the selected probes to generate final HCR probe pairs

In [9]:
hcr_probes = add_hairpin(probe_datasheet,'B5')

Generating HCR probes...
HCR probes designed


View final HCR probes

In [10]:
hcr_probes.head()

,Oligo1_Position,Oligo2_Position,Oligo1_Sequence,Oligo2_Sequence,Score (average hits),HCRprobe1,HCRprobe2
0,23,24,TATCATATTCCAAAATTGCAACTAC,TGCACTAACTCCACGTCGACTTAAA,1.0,CTCACTCCCAATCTCTATAATATCATATTCCAAAATTGCAACTAC,TGCACTAACTCCACGTCGACTTAAAAACTACCCTACAAATCCAAT
1,35,36,TAAATTTTGATAACTATTCGAAGCA,TGGCGTATTTGAACCAGTGCCTGGT,1.0,CTCACTCCCAATCTCTATAATAAATTTTGATAACTATTCGAAGCA,TGGCGTATTTGAACCAGTGCCTGGTAACTACCCTACAAATCCAAT
2,41,42,GACAGGGCCGTTGTATGGTCCTCCT,TGGACCTACTGCTACAGGGCCACCT,1.0,CTCACTCCCAATCTCTATAAGACAGGGCCGTTGTATGGTCCTCCT,TGGACCTACTGCTACAGGGCCACCTAACTACCCTACAAATCCAAT
3,84,85,ATCTTCATCCCCAAACTCACCTCCC,GTTTTCAAGCCTAGTGATCAATCGT,1.0,CTCACTCCCAATCTCTATAAATCTTCATCCCCAAACTCACCTCCC,GTTTTCAAGCCTAGTGATCAATCGTAACTACCCTACAAATCCAAT
4,89,90,TCCTCCGCTCATTGGACCTCCATCA,TCCTCGATCCACCTGCCATGAAGTA,1.0,CTCACTCCCAATCTCTATAATCCTCCGCTCATTGGACCTCCATCA,TCCTCGATCCACCTGCCATGAAGTAAACTACCCTACAAATCCAAT


The hcr_probes datasheet has the following columns:

'Oligo1_position' and 'Oligo2_position' columns represent the location of the oligo along the transcript

'Oligo1_Sequence' and 'Oligo2_Sequence' are reverse complement oligos of the two neighbouring 25 base sequence along the transcript

'Score (avg hits)' represents how many other transcripts on average could partially bind to one of the two oligos, but not both. Lower the score, higher the specificity of the probe pair

'HCRprobe1' and 'HCRprobe2' are the final hairpin appended oligos that should be ordered as ssDNA.

In [ ]:
hcr_probes.to_csv('test_gene_HCRprobes_B1.csv')